# Decision Trees & Random Forests

In this notebook, we'll take a look at one of the most common classification algorithms:  *Random Forests*.  But in order to understand it, we need to start with a different algorithm called a *Decision Tree*.  Let's look at the name:

* `Tree` - The algorithm takes a look at all the features in the dataset, and creates a *tree* - very similiar to a flowchart.  At each internal node in the tree there is the name of a feature.  The leaves are labeled with a class:

![image](https://miro.medium.com/max/1680/0*DKVni_-q7dAKVel7.png)

* `Decision` - The algorithm starts at the top of the tree, and looks at feature name.  Based on the value of the feature, it advances to one of its children.  It then repeats the process until it reaches a leaf node.  This leaf node determines the class.  

What we would like the algorithm to do is to figure out which features need to be at what level of the tree, as well as what the values should be that lead to each child.

Interestingly, the decision tree makes very little assumptions about its data - primarily, it expects the data to be categorical (i.e., one of $n$ possible values).  If the data is instead numerical, it is first discretized into as many groups as needed for the tree.

Let's see an example.


## Example
### The Wine Dataset

In [ ]:
%pylab inline
import pandas as pd

from sklearn.datasets import load_wine

rcParams['figure.figsize'] = (20,7)


wine = load_wine()

In [ ]:
# Create dataframe with features
df = pd.DataFrame(wine['data'], columns=wine['feature_names'])

# Add target
df['class'] = wine['target']

print(df.head())

print(wine['DESCR'])

We've already seen this dataset in the last lesson.  This time around, we'll try to use its features to predict which of 3 *classes* each wine belongs to.  (For example, a class decided by a human expert).  As always, we begin by looking at the data.  In this case, we'll let's see how many instances we have of each class.

In [ ]:
df['class'].value_counts().sort_index().plot(kind='pie')

It looks like we have a fairly similiar number of instances of each class.  This is what is known as a *balanced* data set - where one class does not completely overwhelm other classes.  Many real-world datasets are *imbalanced* - in those cases, special care needs to be taken.

### Setting Up the Experiment

As this is a *supervised* problem, we can split the data into train and test sets.  Note, however, that we need to make sure that both sets have the same distribution of classes - otherwise, we might only see a certain class in the training or in the test sets.  We call this a *stratified* split.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[df.columns[:-1]], df[df.columns[-1]], stratify=df['class'],
                                                   random_state=234234)
print("Training set size: {} {}".format(X_train.shape, y_train.shape))
print("Test set size: {} {}".format(X_test.shape, y_test.shape))

In [ ]:
print("Training set label distribution:")
print(y_train.value_counts().sort_index() / y_train.sum())

print("Test set label distribution:")
print(y_test.value_counts().sort_index() / y_test.sum())

OK then, percentages seem to be similiar.  We can hope that our model will generalize properly.  Let's go ahead and try to fit it.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

print("Accuracy on train set:")
print(accuracy_score(y_train, tree.predict(X_train)))

What do test set results look like?

In [ ]:
print("Accuracy on test set:")
predictions = tree.predict(X_test)
print(accuracy_score(y_test, predictions))

Not a bad score!  But since the dataset is not *perfectly* balanced let's take a look at a more detailed form of errors.  This is called a *confusion matrix*:  values in *rows* are the *actual* (true) values, while values in *columns* are *predicted* values.  Let's look at the confusion matrix for our test set:

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)
cm



So cell (0,0) (top-left corner) means we have 14 values that were correctly identified as class_0.  Cell (0,1) means that 1 class_0 value was mis-classified as class_1, and cell (0,2) means that no class_0 values were misclassified as class_2.  So basically, the bigger the diagonal of the confusion matrix is with regards to the non-diagonal values, the better.  Let's see this in percentages:

In [ ]:
cm / cm.sum(axis=1)

And now in graphical form:

In [ ]:
imshow(cm / cm.sum(axis=1), extent=None)
colorbar()

### Better Metrics

From the above confusion matrix we can see that there 2 forms of possible errors for a single datum:  either it was supposed to be class $i$ but was classified as class $j$ - a *false negative* - or it was supposed to be class $j$ but was classified as class $i$ - a *false positive*.  We would like to know what kind of mistakes our model makes, as often real-world costs differ between false negatives and false positives.

The `precision` is a measure of false positives.  Its formula:

$$ precision = \frac{tp}{tp+fp} $$

The true positives is the number in cell $(i,i)$ of the confusion matrix.  False positives are items *predicted* to be class $i$, but are *actually* class $j \ne i$ - so they are in *column* $i$ but *rows* different from $i$.  In our case, for class 0:

In [ ]:
cm[0,0] / (cm[0,0] + cm[1,0] + cm[2,0])

Hence, precision for class_0 in our model is 0.875 (or 0.88, rounded to 2 decimal points).  In a similiar vein, we'd like to measure the tendency for false negatives.  This is `recall`:

$$ recall = \frac{tp}{tp+fn} $$

False negatives are items that are *actually* class $i$ but *predicted* to be class $j \ne i$ - so they are in *row* $i$ but *columns* different to $i.  In our case for class 0:

In [ ]:
cm[0,0] / (cm[0,0] + cm[0,1] + cm[0,2])

So recall for class_0 in our model is 0.9333 (or 0.93, rounded to 2 decimal points). Obviously, we don't need to calculate this on our own: 

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

The F1 score is the *harmonic mean* of precision and recall:

$$ F_1 = 2 \cdot \frac{precision \cdot recall}{precision + recall} $$

## The Random Forest Algorithm

If you recall, we saw in our earlier lessons that a single train-test split might not be representative and in this way lead to overfitting.  We also saw the technique of *k-fold cross-validation* that makes sure every individual piece of data appears once in the test set and $k-1$ time in the train set, for a given $k$.  

The *Random Forests* algorithm does something similiar for decision trees.  Rather than use a single decision tree and (over)-fit it to specific data, it fits *many* trees to different subsets of both data *and* features.  This idea is called *bagging* and its purpose is to provide many *weak* models (i.e., trained on only partial data) that can be combined into a *strong* model.  I.e., for every new piece of data, each one of those $n$ models can give its 'opinion' and the final result is determined by a majority vote.

Let's see this in action:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=10)
random_forest.fit(X_train, y_train)

print(classification_report(y_train, random_forest.predict(X_train)))

In [ ]:
print(classification_report(y_test, random_forest.predict(X_test)))

So a marked improvement from the single-tree F1 score!  Let's see how far this idea can take us:

In [ ]:
from sklearn.metrics import f1_score

scores = []

for num_trees in range(1,100):
    forest = RandomForestClassifier(n_estimators=num_trees, random_state=65465)
    forest.fit(X_train, y_train)
    scores.append(f1_score(y_test, forest.predict(X_test), average="weighted"))
    

plot(range(1,100), scores)
grid('both')
xlabel("# of Tress in Random Forest")
ylabel("$F_1$ score")    

## Summary

In this notebook we saw two variants of decision trees - a single tree trained on all data and features, and a forest of trees trained on *random* subsets of data and features.  We discussed the concepts of precision and recall and used them to measure the tendency of our model to produce false negatives and false positives.  Finally, we saw that even a relatively small random forest can already lead to good models.

 ## Exercise

1.  From the Machine Learning Repository at the University of California at Irvine (https://archive.ics.uci.edu/ml/datasets) choose any dataset that has the classification task.  What is F1 score for a single decision tree?  For a random forest with n=10 trees?  What is the optimal number of trees for the model?